In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, LSTM, Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train_df = pd.read_csv(r'C:\Users\Lenovo\Desktop\train.csv')
test_df = pd.read_csv(r'C:\Users\Lenovo\Desktop\test.csv')

In [ ]:
X_train = train_df.iloc[:, :-2].values  
y_train = train_df.iloc[:, -1].values   

X_test = test_df.iloc[:, :-2].values
y_test = test_df.iloc[:, -1].values

In [ ]:
def plot_sensor_data(activity_label, data):
    fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(15, 10))
    axes = axes.flatten()
    activities = np.unique(y_train)
    
    for i, ax in enumerate(axes):
        ax.plot(data[y_train == activity_label][:, i])
        ax.set_title(f'Activity {activity_label} - Sensor {i+1}')
        ax.set_yticks([])
    
    plt.tight_layout()
    plt.show()

for activity_label in np.unique(y_train):
    plot_sensor_data(activity_label, X_train)

In [ ]:
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
import numpy as np

def sliding_window(data, labels, window_size, overlap):
    segments = []
    segment_labels = []
    
    for start in range(0, len(data), overlap):
        end = start + window_size
        if end >= len(data):
            break
        segment = data[start:end]
        segment_label = labels[start:end]
        segments.append(segment)
        segment_labels.append(segment_label)
    
    return np.array(segments), np.array(segment_labels)

window_size = 128   
overlap = 64        

X_train_windowed, y_train_windowed = sliding_window(X_train, y_train, window_size, overlap)
X_test_windowed, y_test_windowed = sliding_window(X_test, y_test, window_size, overlap)

X_train_windowed = X_train_windowed.reshape(-1, window_size, X_train.shape[1])
X_test_windowed = X_test_windowed.reshape(-1, window_size, X_test.shape[1])

y_train_windowed = y_train_windowed[:X_train_windowed.shape[0]]
y_test_windowed = y_test_windowed[:X_test_windowed.shape[0]]

print(f"X_train_windowed shape: {X_train_windowed.shape}")
print(f"y_train_windowed shape: {y_train_windowed.shape}")
print(f"X_test_windowed shape: {X_test_windowed.shape}")
print(f"y_test_windowed shape: {y_test_windowed.shape}")

In [ ]:
X_train_flat = X_train_windowed.reshape(-1, X_train_windowed.shape[2])
X_test_flat = X_test_windowed.reshape(-1, X_test_windowed.shape[2])
y_train_flat = y_train_windowed.reshape(-1)
y_test_flat = y_test_windowed.reshape(-1)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, precision_recall_fscore_support

rf_classifier = RandomForestClassifier(random_state=42)

rf_classifier.fit(X_train_flat, y_train_flat)
y_pred_rf = rf_classifier.predict(X_test_flat)

accuracy_rf = accuracy_score(y_test_flat, y_pred_rf)
print(f"Random Forest Accuracy: {accuracy_rf:.4f}")

print(classification_report(y_test_flat, y_pred_rf))

precision, recall, fscore, _ = precision_recall_fscore_support(y_test_flat, y_pred_rf, average='weighted')
print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {fscore:.4f}")


In [ ]:
from sklearn.svm import SVC

svm_classifier = SVC(kernel='linear', random_state=42)

svm_classifier.fit(X_train_flat, y_train_flat)
y_pred_svm = svm_classifier.predict(X_test_flat)

accuracy_svm = accuracy_score(y_test_flat, y_pred_svm)
print(f"SVM Accuracy: {accuracy_svm:.4f}")

print(classification_report(y_test_flat, y_pred_svm))


precision, recall, fscore, _ = precision_recall_fscore_support(y_test_flat, y_pred_svm, average='weighted')
print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {fscore:.4f}")


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn_classifier = KNeighborsClassifier()

knn_classifier.fit(X_train_flat, y_train_flat)
y_pred_knn = knn_classifier.predict(X_test_flat)


accuracy_knn = accuracy_score(y_test_flat, y_pred_knn)
print(f"KNN Accuracy: {accuracy_knn:.4f}")

print(classification_report(y_test_flat, y_pred_knn))

precision, recall, fscore, _ = precision_recall_fscore_support(y_test_flat, y_pred_knn, average='weighted')
print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {fscore:.4f}")


In [ ]:
from sklearn.model_selection import GridSearchCV

rf_classifier = RandomForestClassifier(random_state=42)

param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search_rf = GridSearchCV(estimator=rf_classifier, param_grid=param_grid_rf, cv=5, scoring='accuracy', verbose=1)

grid_search_rf.fit(X_train_flat, y_train_flat)
best_params_rf = grid_search_rf.best_params_
best_score_rf = grid_search_rf.best_score_
print(f"Best Parameters for Random Forest: {best_params_rf}")
print(f"Best Accuracy Score for Random Forest: {best_score_rf:.4f}")

best_rf_model = grid_search_rf.best_estimator_
y_pred_rf = best_rf_model.predict(X_test_flat)
accuracy_rf = accuracy_score(y_test_flat, y_pred_rf)
print(f"Random Forest Accuracy: {accuracy_rf:.4f}")

print(classification_report(y_test_flat, y_pred_rf))

precision, recall, fscore, _ = precision_recall_fscore_support(y_test_flat, y_pred_rf, average='weighted')
print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {fscore:.4f}")

In [ ]:
from sklearn.svm import SVC

param_grid_svm = {
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto'],
    'kernel': ['linear', 'rbf', 'poly']
}

svm_classifier = SVC(random_state=42)

grid_search_svm = GridSearchCV(estimator=svm_classifier, param_grid=param_grid_svm, cv=5, scoring='accuracy', verbose=1)

grid_search_svm.fit(X_train_flat, y_train_flat)
best_params_svm = grid_search_svm.best_params_
best_score_svm = grid_search_svm.best_score_
print(f"Best Parameters for SVM: {best_params_svm}")
print(f"Best Accuracy Score for SVM: {best_score_svm:.4f}")

best_svm_model = grid_search_svm.best_estimator_
y_pred_svm = best_svm_model.predict(X_test_flat)

accuracy_svm = accuracy_score(y_test_flat, y_pred_svm)
print(f"SVM Accuracy: {accuracy_svm:.4f}")

print(classification_report(y_test_flat, y_pred_svm))

precision, recall, fscore, _ = precision_recall_fscore_support(y_test_flat, y_pred_svm, average='weighted')
print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {fscore:.4f}")


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

param_grid_knn = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski']
}

knn_classifier = KNeighborsClassifier()

grid_search_knn = GridSearchCV(estimator=knn_classifier, param_grid=param_grid_knn, cv=5, scoring='accuracy', verbose=1)

grid_search_knn.fit(X_train_flat, y_train_flat)
best_params_knn = grid_search_knn.best_params_
best_score_knn = grid_search_knn.best_score_
print(f"Best Parameters for KNN: {best_params_knn}")
print(f"Best Accuracy Score for KNN: {best_score_knn:.4f}")

best_knn_model = grid_search_knn.best_estimator_
y_pred_knn = best_knn_model.predict(X_test_flat)

accuracy_knn = accuracy_score(y_test_flat, y_pred_knn)
print(f"KNN Accuracy: {accuracy_knn:.4f}")

print(classification_report(y_test_flat, y_pred_knn))

precision, recall, fscore, _ = precision_recall_fscore_support(y_test_flat, y_pred_knn, average='weighted')
print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {fscore:.4f}")
